Paper: https://arxiv.org/pdf/1601.01178.pdf

# Notation

### Bookkeeping symbols (stored/set per node)

| Symbol | Description   |
|------|------|
| $\rho$ | number of conducted trials in negative binomial process |
| $t$ | flag denoting whether or not this is the end of a game |
| $d$ | depth of node in tree (number of moves made in the game) |
| $\psi$ | number of siblings |
| $k$ | number of children |
| $\varphi$ | controls the trade off between variance of child means and variance of child stddevs |
| $p$ | vector on $\mathbb{R}^k$ denoting the likelihood of visiting each child node |
| $\mu$ | The mean of the node |
| $\sigma$ | The stddev of the node |

### Hyperparameters (to be learned)

| Symbol | Description |
| ------ | ----------- |
| $p_{t}$ | probability of success in negative binomial process|
| $p_{\text{pos}}$ | probability of drawing positive noise in $k$ random walk |
| $\lambda_{\varepsilon}$ | characterizes Poisson distribution for sampling noise in $k$ random walk |
| $\omega$ | constant drift in $k$ random walk |
| $r$ | maximum number of trials in negative binomial process |
| $f$ | characterizes a Beta distribution to use for sampling a $\varphi^2$ at a given node|
| $g$ | gives a concentration scale factor, $c$ to use when sampling child standard deviations |

### Hyperparameters (to be set directly)
| Symbol | Description |
| ------ | ----------- |
| $\mu_0$ | The mean of the root node |
| $\sigma_0$ | The stddev of the root node |
| $k_0$ | The number of children of the root node |
### Misc

| Symbol | Description |
| -------- | ----------- |
| $b(p)$ | function which gives an orthornormal basis for the hyperplane orthogonal to $p$ |

# Algorithm for node creation

#### 1. Set terminal flag
$\epsilon \sim NB(1, p_t)$

$\rho \gets \rho + \epsilon + 1$

$t = \begin{cases} 0 & \rho \leq r \\ 1 & \text{otherwise} \end{cases} $

#### 2. Set number of children (k)
$\pi \sim \text{Bernoulli}(p_{\text{pos}})$

$\phi = \begin{cases}
    -1 & \pi = 0 \\
    1 & \pi = 1
  \end{cases}
$

$\varepsilon \sim \text{Poisson}(\lambda_\varepsilon)$

$\delta = \phi\varepsilon + \omega$

$k = \begin{cases} 
      0 & t = 1\\
      \psi + \delta & \text{otherwise}
   \end{cases}
$

#### 3. Sample a finite mixture over the child nodes

$p = \mathbb{1}^k\cdot\frac{1}{k}$

Note: I think we could simply use a Dirichlet prior for this.

##### 3a. Sample the dispersion parameter

$\begin{bmatrix}
         \alpha \\
         \beta \\
        \end{bmatrix} = f(\mu_p, \sigma_p, d, k)$
        
$\varphi^2 \sim \text{Beta}(\alpha,\beta)$

##### 3b. Sample the child means

$F = b(p)$

$q \sim N(0,1)^k$

$\gamma = \frac{F^T\cdot q}{|F^T\cdot q|}\varphi$

$\alpha = \frac{\gamma}{\sqrt{p}}$

$\mu_c = \mu + \sigma\alpha$

##### 3c. Sample the child standard deviations

$c = g(\mu, \sigma, d, k, \varphi^2)$

$v \sim \text{Dirichlet}(\mathbb{1}^k \cdot c)$

We use Dirichlet because the only way we can solve for $v$ from observed data is if we know $\sum_iv_i$.

$\eta = \frac{v}{|v|}\sqrt{1 - \varphi^2}$

$\tau = \frac{\eta}{\sqrt{p}}$

$\sigma_c = \sigma\tau$

###### 4. Repeat 1-3 for newly created nodes


